In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import time
import pandas

# increase the width of boxes in the notebook file (this is only cosmetic)
np.set_printoptions(linewidth=180)

# Problem 2

The gradient OLS code from class:

In [2]:
class OrdinaryLeastSquaresGradient:
        
    # fit the model to the data
    def fit(self, X, y, x0, alpha, h, tolerance, maxIterations):
        self.n = X.shape[0]
        self.d = X.shape[1]
        self.h = h
        self.alpha = alpha
        self.initialGuess = x0
        
        # save the training data
        self.data = np.hstack((np.ones([self.n, 1]), X))
        
        # save the training labels
        self.outputs = y
        
        # find the beta values that minimize the sum of squared errors via gradient descent
        X = self.data
        L = lambda beta: ((X @ beta).T - y.T) @ (X @ beta - y)
        # self.beta = self.gradientDescent(L,(self.d +  1) * [0], h, tolerance, maxIterations)
        self.beta = self.gradientDescent(L, self.initialGuess, self.alpha, self.h, tolerance, maxIterations)
                
    # predict the output from testing data
    def predict(self, X):
        # initialize an empty matrix to store the predicted outputs
        yPredicted = np.empty([X.shape[0],1])
        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # apply the function f with the values of beta from the fit function to each testing datapoint (rows of X)
        for row in range(X.shape[0]):
            yPredicted[row] = self.beta @ X[row,]
            
        return yPredicted

    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, alpha, h, tolerance, maxIterations):
        # set x equal to the initial guess
        x = x0

        # take up to maxIterations number of steps
        for counter in range(maxIterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)

            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                # return the approximate critical value x
                return x

            # if we do not converge, print a message
            elif counter == maxIterations-1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
            
    # compute the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)
        fx = f(x)

        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - fx)/h

        return gradient

The gradient method for OLS with momentum:

In [3]:
class OrdinaryLeastSquaresGradientMomentum:
        
    # fit the model to the data
    def fit(self, X, y, x0, alpha, gamma, h, tolerance, maxIterations):
        self.n = X.shape[0]
        self.d = X.shape[1]
        self.h = h
        self.alpha = alpha
        self.gamma = gamma
        self.initialGuess = x0
        
        # save the training data
        self.data = np.hstack((np.ones([self.n, 1]), X))
        
        # save the training labels
        self.outputs = y
        
        # find the beta values that minimize the sum of squared errors via gradient descent
        X = self.data
        L = lambda beta: ((X @ beta).T - y.T) @ (X @ beta - y)
        # self.beta = self.gradientDescent(L,(self.d +  1) * [0], h, tolerance, maxIterations)
        self.beta = self.gradientDescent(L, self.initialGuess, self.alpha, self.gamma, self.h, tolerance, maxIterations)
                
    # predict the output from testing data
    def predict(self, X):
        # initialize an empty matrix to store the predicted outputs
        yPredicted = np.empty([X.shape[0],1])
        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # apply the function f with the values of beta from the fit function to each testing datapoint (rows of X)
        for row in range(X.shape[0]):
            yPredicted[row] = self.beta @ X[row,]
            
        return yPredicted

    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, alpha, gamma, h, tolerance, maxIterations):
        # set x equal to the initial guess
        x = x0
        v = 0

        # take up to maxIterations number of steps
        for counter in range(maxIterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)

            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                # return the approximate critical value x
                return x

            # if we do not converge, print a message
            elif counter == maxIterations-1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            v = gamma*v + alpha*gradient
            x -= v
            
    # compute the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)
        fx = f(x)

        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - fx)/h

        return gradient

Let's run both

In [8]:
# import the data from the csv file to an numpy array
data = pandas.read_csv('Mount_Pleasant_Real_Estate_Data.csv', sep=',').to_numpy()

X = np.array(data[:,2:-1], dtype=float)
y = np.array(data[:,1], dtype=float)

# split the data into training and test sets
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size = 0.25, random_state = 1)

trainX = scale(trainX)
testX = scale(testX)

print('FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE \n')

startTime = time.time()

# instantiate an OLS model
model = OrdinaryLeastSquaresGradient()

# fit the model to the training data (find the beta parameters)
model.fit(trainX, trainY, np.zeros(31), alpha = 0.0001, h = 0.0001, tolerance = 0.01, maxIterations = 100000)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)

# print the coefficient of determination r^2
print('\nThe r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics
# print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

# print the beta values
# print('The beta values are', model.beta)

# print quality metrics
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions), '\n')

print('The time elapsed was', time.time() - startTime, '\n')

print('FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE WITH MOMENTUM \n')

startTime = time.time()

# instantiate an OLS model
model = OrdinaryLeastSquaresGradientMomentum()

# fit the model to the training data (find the beta parameters)
model.fit(trainX, trainY, np.zeros(31), alpha = 0.00001, gamma = 0.9, h = 0.00001, tolerance = 0.01, maxIterations = 100000)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)

# print the coefficient of determination r^2
print('\nThe r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics
# print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

# print the beta values
# print('The beta values are', model.beta)

# print quality metrics
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions), '\n')

print('The time elapsed was', time.time() - startTime)

FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE 

Gradient descent took 5976 iterations to converge
The norm of the gradient is 0.0

The r^2 score is 0.8383446613817547
The mean absolute error on the test set is 102985.33347096098 

The time elapsed was 1.9966447353363037 

FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE WITH MOMENTUM 

Gradient descent took 2880 iterations to converge
The norm of the gradient is 0.0

The r^2 score is 0.8383446613815551
The mean absolute error on the test set is 102985.32709164586 

The time elapsed was 1.0202727317810059


For the gradient method, we get mean absolute error 102985 on the test set and it runs in 2.04 seconds. With momentum, we get roughly the same accuracy, but it runs in only 0.94 seconds.

# Problem 3

For elastic net, we need a similar code to the previous problem, except the loss function will have $L^1$ and $L^2$ penalties on the parameters $\beta$ with corresponding hyperparameters $\lambda_1$ and $\lambda_2$. Here's the code:

In [5]:
class ElasticNetGradientMomentum:
        
    # fit the model to the data
    def fit(self, X, y, x0, alpha, gamma, lambda1, lambda2, h, tolerance, maxIterations):
        self.n = X.shape[0]
        self.d = X.shape[1]
        self.h = h
        self.alpha = alpha
        self.gamma = gamma
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.initialGuess = x0
        
        # save the training data
        self.data = np.hstack((np.ones([self.n, 1]), X))
        
        # save the training labels
        self.outputs = y
        
        # find the beta values that minimize the sum of squared errors via gradient descent
        X = self.data
        L = lambda beta: ((X @ beta).T - y.T) @ (X @ beta - y) + self.lambda1 * beta.T @ beta + self.lambda2 * np.sum(np.abs(beta))
        # self.beta = self.gradientDescent(L,(self.d +  1) * [0], h, tolerance, maxIterations)
        self.beta = self.gradientDescent(L, self.initialGuess, self.alpha, self.gamma, self.lambda1, self.lambda2,
                                         self.h, tolerance, maxIterations)
                
    # predict the output from testing data
    def predict(self, X):
        # initialize an empty matrix to store the predicted outputs
        yPredicted = np.empty([X.shape[0],1])
        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # apply the function f with the values of beta from the fit function to each testing datapoint (rows of X)
        for row in range(X.shape[0]):
            yPredicted[row] = self.beta @ X[row,]
            
        return yPredicted

    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, alpha, gamma, lambda1, lambda2, h, tolerance, maxIterations):
        # set x equal to the initial guess
        x = x0
        v = 0

        # take up to maxIterations number of steps
        for counter in range(maxIterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)

            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                # return the approximate critical value x
                return x

            # if we do not converge, print a message
            elif counter == maxIterations-1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            v = gamma*v + alpha*gradient
            x -= v
            
    # compute the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)
        fx = f(x)

        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - fx)/h

        return gradient

# Problem 4

In [6]:
print('FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE WITH MOMENTUM \n')

startTime = time.time()

# instantiate an OLS model
model = ElasticNetGradientMomentum()

# fit the model to the training data (find the beta parameters)
model.fit(trainX, trainY, np.zeros(31), alpha = 0.0001, gamma = 0, lambda1 = 0.2, lambda2 = 0.4, h = 0.0001, tolerance = 0.01, maxIterations = 1000)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)

# print the coefficient of determination r^2
print('\nThe r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

# print the beta values
# print('The beta values are', model.beta)

# print quality metrics
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions), '\n')

print('The time elapsed was', time.time() - startTime)

FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE WITH MOMENTUM 

Gradient descent failed
The gradient is [    0.          903.3203125  6992.1875      175.78125     122.0703125   205.078125   1313.4765625 -1865.234375  -1982.421875   3364.2578125 -9111.328125     92.7734375
  3833.0078125   839.84375      97.65625    1372.0703125   170.8984375  -405.2734375  -439.453125    239.2578125   263.671875   -302.734375    234.375      -463.8671875
     0.         1982.421875      0.            0.         1147.4609375  -297.8515625 -6025.390625 ]

The r^2 score is 0.8383372951895538
The mean absolute error on the training set is 88233.7453366127
The mean absolute error on the test set is 102920.50090670281 

The time elapsed was 0.6871771812438965
